## 流量を管理するモジュール

#### 重力加速度

In [1]:
def grav():
    return 9.806650

### 熱伝導（フーリエの法則）  
定義：$\dot q = -\lambda\nabla・T$  
$\dot q$：単位時間当たりの熱流量の密度[W/m2]  
$\lambda$：熱伝導率[W/mK]  
$T$：絶対温度[K]  

#### 差分化その1：$\dot q = -\lambda_{ave.}\frac{T_{l+1}-T_{l}}{dx_{l+1}+dx_{l}}$  
$\lambda_{ave.}$：平均の熱伝導率  
$l, l+1$：ある点のセル・隣接する点のセル  
$dx$：セルの質点からセル境界までの距離

物性値の調和平均：
$\lambda_{ave}=\frac{\lambda_{l+1} dx_{l+1} + \lambda_{l} dx_{l} }{ dx_{l+1} + dx_{l} }$  
物性値を材料の長さに応じて調和平均化する。熱伝導率の差が小さい時は問題ないが、差が大きくなるとその2のように熱抵抗の和として考えるのが妥当

In [2]:
def mean_average( val_mns, val_pls, len_mns, len_pls ):
    return ( val_mns * len_mns + val_pls * len_pls ) / ( len_mns + len_pls )

In [3]:
def heat_conduction_v1( lamL1, lamL, tempL1, tempL, dxL1, dxL ):
    lamAve = mean_average( val_mns = lamL, val_pls = lamL1, len_mns = dxL, len_pls = dxL1 )
    return - lamAve * ( tempL1 - tempL ) / ( dxL1 + dxL )

In [5]:
#　使用例
heat_conduction_v1( 0.1, 1.0, 20.0, 22.0, 0.01, 0.01 )

54.99999999999999

#### 差分化その2：$\dot q = -K(T_{l+1}-T_{l})$  
$K$：熱貫流率[W/K]  
一般的にはこちらを採用。異種材料間の熱の移動は熱抵抗値の和として計算。

熱貫流率：
$K = \frac{ 1 } { \frac{ dx_{l+1} } { \lambda_{l+1} } + \frac{ dx_{l} }{ \lambda_{l} } } $  

In [7]:
def transmittance( val_mns, val_pls, len_mns, len_pls ):
    return 1.0 / ( len_mns / val_mns + len_pls / val_pls )

In [8]:
def heat_conduction_v2( lamL1, lamL, tempL1, tempL, dxL1, dxL ):
    K = transmittance( val_mns = lamL, val_pls = lamL1, len_mns = dxL, len_pls = dxL1 )
    return - K * ( tempL1 - tempL )

In [9]:
#　使用例
heat_conduction_v2( 0.1, 1.0, 20.0, 22.0, 0.01, 0.01 )

18.181818181818183

#### 差分化その3：$\dot q = -\lambda_{ave.}\frac{\Delta T}{\Delta x2}$  
$\Delta T$：質点間の温度差[K]  
$\Delta x2$：質点間の距離[m]  
平均の熱伝導率、温度差などをあらかじめ計算している場合はこちらを用いる。

In [11]:
def heat_conduction_v3( lam, dtemp, dx2 ):
    return - lam * dtemp / dx2

In [12]:
# 使用例  
heat_conduction_v3( 9.090, -2.0, 1.0 )

18.18

### 熱伝達  
定義：$\dot q = \alpha(T_{air} - T_{wall})$  
空気に接する壁表面には空気の熱伝達層が存在すると考えた場合の熱流。空気から壁表面への熱流を正とした。  
$\dot q$：単位時間当たりの熱流量の密度[W/m2]  
$\alpha$：熱伝達率[W/m2K]  
$T_{air}$：空気の絶対温度[K]  
$T_{wall}$：壁表面の絶対温度[K]  

In [13]:
def heat_transfer( alpha, temp_air, temp_wall ):
    return alpha * ( temp_air - temp_wall )

In [14]:
# 使用例
heat_transfer( 9.3, 20.0, 15.0 )

46.5

## 水蒸気移動  
元原理：Fickの拡散法則

### 水蒸気移動（水蒸気圧勾配・固体内）  
定義：$J_v = -\lambda^{'}_{p}\nabla・P_v$  
$J_v$：気相水分流量[kg/m2s]  
$\lambda^{'}_{p}$：水蒸気圧勾配に関する気相水分伝導率[kg/msPa]  
$P_v$：水蒸気圧[Pa]  

In [12]:
def vapourTransfer_pressure( ldp, dpv, dx2 ):
    return - ldp * dpv /dx2

## 液水移動  
原理：ダルシー則

### 液水移動（水分化学ポテンシャル勾配・固体内）  
定義：$J_l = -\lambda^{'}_{\mu_l}(\nabla・\mu - n_x g)$    
$J_l$：液相水分流量[kg/m2s]  
$\lambda^{'}_{\mu l}$：水分化学ポテンシャル勾配に関する液相水分伝導率[kg/ms(J/kg)]  
$\mu$：液水の化学ポテンシャル[J/kg]  
$n_x$：重力加速度に対する応答（重力加速度に対して水平なら1、垂直なら0）

In [13]:
def liquidTransfer_potential( ldml, dmiu, dx2, nx ):
    return - ldml * ( dmiu / dx2 - nx * grav() )